See readme.md for ideal fields and descriptions

In [ ]:
# ! pip install pandas pyarrow
# ! pip install polars

In [ ]:
import subprocess
import os
from datetime import datetime, timedelta, timezone
import time
import pyarrow.parquet as pq
import pandas as pd
# test adding cryo_cli python

In [ ]:
import polars as pl

In [ ]:
# Init timestamps
trailing_days = 7

current_date_utc = datetime.utcnow().date()
# Convert the current date to a Unix timestamp (remaining in UTC)
current_timestamp_utc = datetime(current_date_utc.year, current_date_utc.month, current_date_utc.day, tzinfo=timezone.utc).timestamp()
previous_date_utc = current_date_utc - timedelta(days=trailing_days)
previous_timestamp_utc = datetime(previous_date_utc.year, previous_date_utc.month, previous_date_utc.day, tzinfo=timezone.utc).timestamp()

print('end: ' + str(current_date_utc))
print('start: ' + str(previous_date_utc))

In [ ]:
# Test doing Lyra

fields = 'blocks txs'# traces'
rpc_url = 'https://rpc.lyra.finance/'
chain_name = 'lyra'
start_timestamp = int(previous_timestamp_utc)
end_timestamp = int(current_timestamp_utc)

dry_run = 0
requests_per_second_max = 500 # -1 means ignore

In [ ]:
# Generate Command
if dry_run == 1:
    dry_txt = '--dry'
else:
    dry_txt = ''
if requests_per_second_max > -1:
    rps_txt = '--requests-per-second ' + str(requests_per_second_max)
else:
    rps_txt = ''

command = f"cryo {fields} --rpc {rpc_url} --timestamps {start_timestamp}:{end_timestamp} --subdirs datatype --label {chain_name} {rps_txt} {dry_txt}"
print(command)

In [ ]:
start_time = time.time()
# Run the command using subprocess.run and capture the output
result = subprocess.run(
    command, 
    shell=True, 
    stdout=subprocess.PIPE,  # Capture standard output
    stderr=subprocess.PIPE,  # Capture standard error
    text=True  # Capture output as text (Python 3.7+)
)

# Display the captured output
if result.returncode == 0:
    print("Command succeeded. Output:")
    print(result.stdout)
else:
    print("Command failed. Error output:")
    print(result.stderr)

end_time = time.time()

In [ ]:
# Calculate the elapsed time
elapsed_time = end_time - start_time
# Print the elapsed time in seconds
print(f"Elapsed time: {elapsed_time:.4f} seconds")

In [ ]:
# # Read parquet files
txs = pl.scan_parquet('transactions__' + chain_name + '/*.parquet')
blocks = pl.scan_parquet('blocks__' + chain_name + '/*.parquet')

# Rename the 'gas_used' column to 'block_gas_used' in the 'blocks' DataFrame
blocks = blocks.rename({"gas_used": "block_gas_used"})

# Perform the join on 'block_number' and 'chain_id'
joined_df = blocks.join(
    txs,
    on=["block_number", "chain_id"],
    how="inner"  # You can specify the type of join you want (inner, outer, left, right)
)

# Convert Unix timestamp to datetime and create a new column 'timestamp_dt'
joined_df = joined_df.with_columns(
    pl.from_epoch("timestamp", time_unit="s").alias("timestamp_dt")
)

# Truncate the 'timestamp_dt' column to the day and create a new column 'timestamp_date'
joined_df = joined_df.with_columns(
    pl.col("timestamp_dt").dt.truncate("1d").alias("timestamp_date")
)

In [ ]:
# print(blocks.schema)
# print(txs.schema)
print(joined_df.schema)

#test output
joined_pd = joined_df.collect().to_pandas()
joined_pd.tail(5)

print('num blocks: ' + str(joined_pd['block_number'].nunique()))

In [ ]:
# Create a reference list of consecutive block numbers
reference_block_numbers = list(range(joined_pd['block_number'].min(), joined_pd['block_number'].max() + 1))

# Find the missing block numbers
missing_block_numbers = set(reference_block_numbers) - set(joined_pd['block_number'])

print("Missing block numbers:", missing_block_numbers)

# Convert the missing_block_numbers set to a list and sort it
missing_block_numbers_list = sorted(list(missing_block_numbers))
is_consecutive = all(missing_block_numbers_list[i] == missing_block_numbers_list[i - 1] + 1 for i in range(1, len(missing_block_numbers_list)))

if is_consecutive:
    print("The missing block numbers are consecutive.")
else:
    print("The missing block numbers are not consecutive.")

In [ ]:
# Assuming you have a DataFrame named 'joined_df' with the required columns

result_df = joined_df.group_by([pl.col("timestamp_date"), pl.col("chain_id")]).agg(
    num_blocks=pl.col("block_number").n_unique(),
    min_block_number=pl.col("block_number").min(),
    max_block_number=pl.col("block_number").max(),
    min_block_time=pl.col("timestamp").min(),
    max_block_time=pl.col("timestamp").max(),

    num_user_transactions=
        pl.when(pl.col("gas_price") > 0).then(pl.col("transaction_hash")).count(),
    num_success_user_transactions=
        pl.when((pl.col("gas_price") > 0) & pl.col("success")).then(pl.col("transaction_hash")).count(),
    num_senders=pl.col("from_address").filter(pl.col("gas_price") > 0).n_unique(),

    total_gas_used=pl.col("gas_used").sum(),
    user_gas_used=pl.col("gas_used").filter(pl.col("gas_price") > 0).sum(),
    total_gas_used_per_block = pl.col("gas_used").sum() / pl.col("block_number").n_unique(),
    user_gas_used_per_block = pl.col("gas_used").filter(pl.col("gas_price") > 0).sum() / pl.col("block_number").n_unique(),
    
    l2_fees_base_fees_eth=(pl.col("base_fee_per_gas") * pl.col("gas_used")).sum() / 1e18,
    l2_fees_priority_fees_eth=pl.when(pl.col("gas_price") > 0).then((pl.col("gas_price") - pl.col("base_fee_per_gas")) * pl.col("gas_used")).sum() / 1e18,
    l2_fees_total_fees_eth=(pl.col("gas_price") * pl.col("gas_used")).sum() / 1e18,
)
result_df

In [ ]:
#Execute and turn to pandas
result_df = result_df.collect().to_pandas()


In [ ]:
# Filter
result_df = result_df[result_df['min_block_time']>= start_timestamp] #seems like 1 block before gets pulled. yolo.

In [ ]:

result_df['min_block_time_dt'] = pd.to_datetime(result_df['min_block_time'], unit='s')
result_df['max_block_time_dt'] = pd.to_datetime(result_df['max_block_time'], unit='s')
display(result_df.sort_values(by='timestamp_date',ascending=False))